## Chassisml Example Notebooks
Welcome to the examples section for [Chassis.ml](https://chassis.ml), which contains notebooks that leverage Chassisml to auto-containerize models built using the most common machine learning frameworks. 

**NOTE:** Chassisml provides two key functionalities: 
1. Create a Docker container from your model code and push that container image to a Docker registry. This is the default behavior.
2. Should you pass valid Modzy credentials as optional parameters, Chassisml will take the container and upload it directly to the Modzy environment you specify. You will notice most of these notebooks deploy the model to one of the Modzy internal development environments.   

Can't find the framework you are looking for or need help? Fork this repository and open a PR, we're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/tdfXFY2y), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly! 

In [1]:
import chassisml
import numpy as np
import getpass
import json
from sklearn.linear_model import LogisticRegression
from sklearn import datasets

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

docker hub username········
docker hub password········
modzy api key········


## Train model

In [3]:
# Import and normalize data
X_digits, y_digits = datasets.load_digits(return_X_y=True)
X_digits = X_digits / X_digits.max()

n_samples = len(X_digits)

# Split data into training and test sets
X_train = X_digits[: int(0.9 * n_samples)]
y_train = y_digits[: int(0.9 * n_samples)]
X_test = X_digits[int(0.9 * n_samples) :]
y_test = y_digits[int(0.9 * n_samples) :]

# Train Model
logistic = LogisticRegression(max_iter=1000)
print(
    "LogisticRegression mean accuracy score: %f"
    % logistic.fit(X_train, y_train).score(X_test, y_test)
)

# Save small sample input to use for testing later
sample = X_test[:5].tolist()
with open("data/digits_sample.json", 'w') as out:
    json.dump(sample, out)

LogisticRegression mean accuracy score: 0.933333


## Prepare context dict
Initialize anything here that should persist across inference runs

In [9]:
# This will be passed to Chassis:
context = {"model": logistic}

## Write process function

* Must take bytes and context dict as input
* Preprocess bytes, run inference, postprocess model output, return results

In [10]:
def process(input_bytes,context):
    inputs = np.array(json.loads(input_bytes))
    inference_results = context["model"].predict(inputs)
    structured_results = []
    for inference_result in inference_results:
        structured_output = {
            "data": {
                "result": {"classPredictions": [{"class": str(inference_result), "score": str(1)}]}
            }
        }
        structured_results.append(structured_output)
    return structured_results

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [11]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `context` dict containing all variables which should be loaded once and persist across inferences
* Requires `process_fn` defined above

In [12]:
# create Chassis model
chassis_model = chassis_client.create_model(context=context,process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/digits_sample.json'
results = chassis_model.test(sample_filepath)
print(results)

b'[{"data":{"result":{"classPredictions":[{"class":"5","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"2","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"0","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"1","score":"1"}]}}}]'


In [26]:
# manually construct conda environment to pass to Chassis job

# NOTE: if you define your environment manually like this, the "chassisml" package must be included within the pip depedencies
env = {
    "name": "sklearn-chassis",
    "channels": ['conda-forge'],
    "dependencies": [
        "python=3.8.5",
        {
            "pip": [
                "scikit-learn",
                "numpy",
                "chassisml"
            ] 
        }
    ]
}

In [28]:
# test environment and model within Chassis service, must pass filepath here:

# dry run before build
test_env_result = chassis_model.test_env(test_input_path=sample_filepath, conda_env=env)
print(test_env_result)

Starting test job... Ok!
{'model_output': 'Single input prediction:\n\nb\'[{"data":{"result":{"classPredictions":[{"class":"5","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"2","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"8","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"0","score":"1"}]}}},{"data":{"result":{"classPredictions":[{"class":"1","score":"1"}]}}}]\'\n'}


## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, and required Modzy info

In [ ]:
MODZY_URL = "https://demo.modzy.engineering/api"

response = chassis_model.publish(
    model_name="Sklearn Logistic Regression Digits Image Classification",
    model_version="0.0.2",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    conda_env=env,
    modzy_sample_input_path=sample_filepath,
    modzy_api_key=modzy_api_key,
    modzy_url=MODZY_URL
)

In [30]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

In [31]:
if final_status["result"] is not None:
    print("New model URL: {}".format(final_status["result"]["container_url"]))
else:
    print("Chassis job failed \n\n {}".format(final_status))

New model URL: https://demo.modzy.engineering/models/i6ugw0hl96/0.0.2


## Run sample job using Modzy SDK
Submit inference job to our newly-deploy model running on Modzy

In [33]:
from modzy import ApiClient

client = ApiClient(base_url=MODZY_URL, api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

inference_job = client.jobs.submit_file(model_id, model_version, {input_name: sample_filepath})
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
inference_job_results_json = inference_job_result.get_first_outputs()['results.json']
print(inference_job_results_json)

[ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "5",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "2",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "8",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "0",
          "score": "1"
        }
      ]
    }
  }
}), ApiObject({
  "data": {
    "result": {
      "classPredictions": [
        {
          "class": "1",
          "score": "1"
        }
      ]
    }
  }
})]
